In [1]:
import os
import glob
from collections import Counter
import pysam
from pyBioInfo.Range import GRange
from pyBioInfo.Utils import ShiftLoader

# C57BL X DBA

In [2]:
def load_variants(vcf_path, sample_name, variant_type):
    variants = []
    with pysam.VariantFile(vcf_path) as f:
        for record in f:
            chrom = "chr%s" % record.chrom
            gt = record.samples[sample_name]["GT"]
            allele1, allele2 = record.alleles[gt[0]], record.alleles[gt[1]]
            name = "%s/%s" % (allele1, allele2)
            obj = GRange(chrom=chrom, start=record.start, end=record.stop, name=name, strand="+")
            obj.ref = record.ref
            obj.allele1 = allele1
            obj.allele2 = allele2
            obj.type = variant_type
            obj.filter = list(record.filter)
            variants.append(obj)
    variants.sort()
    return variants

def scan_self_overlap(variants):
    loader = ShiftLoader(variants)
    for x in variants:
        x.overlaps = []
        for x2 in loader.fetch(obj=x):
            if x2 is not x:
                x.overlaps.append(x2)
    return variants

In [3]:
sizes = "stratifications/genomes.sizes"
! sort -k1,1 /home/chenzonggui/species/mus_musculus/GRCm38.p6/GRCm38.canonical.genome.sizes > {sizes}

In [14]:
! zcat ../GRCm38_C57DBA_SV/stratifications/GRCm38_LowComplexity.bed.gz | awk '$3-$2>200' \
    | bedtools slop -i - -g {sizes} -b 50 \
    | bedtools merge -d 50 -i - > stratifications/GRCm38_LowComplexity_gt200bp_slop50bp.bed

! zcat ../GRCm38_C57DBA_SV/stratifications/GRCm38_SimpleRepeats.bed.gz | awk '$3-$2>200' \
    | bedtools slop -i - -g {sizes} -b 50 \
    | bedtools merge -d 50 -i - > stratifications/GRCm38_SimpleRepeats_gt200bp_slop50bp.bed

! zcat ../GRCm38_C57DBA_SV/GRCm38_DBA_ONT/GRCm38.tandemRepeats.bed.gz | awk '$3-$2>200' \
    | bedtools slop -i - -g {sizes} -b 50 \
    | bedtools merge -d 50 -i - > stratifications/GRCm38_TandemRepeats_gt200bp_slop50bp.bed

In [ ]:
with pysam.VariantFile("../GRCm38_C57DBA_SV/GRCm38_C57DBA_SV_benchmark_callsets.vcf.gz") as f,\
    open("stratifications/GRCm38_DBA_SV.bed", "w+") as fw:
    for x in f:
        svlen = abs(x.info["SVLEN"])
        if svlen >= 50:
            fw.write("\t".join(map(str, [x.chrom, x.start, x.stop])) + "\n")

! cat stratifications/GRCm38_DBA_SV.bed | sort -k1,1 -k2,2n \
    | bedtools slop -i - -g {sizes} -b 500 \
    | bedtools merge -d 100 -i - > stratifications/GRCm38_DBA_SV_slop500bp.bed

In [5]:
! bedtools slop -i ../GRCm38_C57DBA_SV/GRCm38_DBA_ONT/GRCm38_DBA_ONT.extreme.width_200bp_prop_0.01_0.99.bed -g {sizes} -b 200 \
    | bedtools merge -d 200 -i - > stratifications/GRCm38_DBA_ExtremeRegion_200bp_slop200bp.bed

! bedtools slop -i ../GRCm38_C57DBA_SV/GRCm38_DBA_ONT/GRCm38_DBA_ONT.extreme.width_500bp_prop_0.01_0.99.bed -g {sizes} -b 500 \
    | bedtools merge -d 200 -i - > stratifications/GRCm38_DBA_ExtremeRegion_500bp_slop500bp.bed

! bedtools slop -i ../GRCm38_C57DBA_SV/GRCm38_DBA_ONT/GRCm38_DBA_ONT.extreme.width_1000bp_prop_0.01_0.99.bed -g {sizes} -b 1000 \
    | bedtools merge -d 200 -i - > stratifications/GRCm38_DBA_ExtremeRegion_1000bp_slop1000bp.bed

## 1. C57BL/6J X DBA/2J

C57BL/6J is GRCm38

In [3]:
variants1 = load_variants("data/DBA_2J.mgp.v5.snps.dbSNP142.vcf.gz", "DBA_2J", "snp")
variants2 = load_variants("data/DBA_2J.mgp.v5.indels.dbSNP142.normed.vcf.gz", "DBA_2J", "indel")
variants = list(sorted(variants1 + variants2))
variants = scan_self_overlap(variants)

In [9]:
outfile = "GRCm38_C57BL_6J_DBA_2J_SNP_Indel.vcf"
records_failed = []
with pysam.VariantFile("data/DBA_2J.mgp.v5.snps.dbSNP142.vcf.gz") as f, open(outfile, "w+") as fw:
    fw.write("##fileformat=VCFv4.2\n")
    fw.write("##FORMAT=<ID=GT,Number=1,Type=String,Description=\"Genotype\">\n")
    fw.write("##FORMAT=<ID=PS,Number=1,Type=String,Description=\"Phase set for GT\">\n")
    for chrom in f.header.contigs:
        if chrom == "MT":
            continue
        fw.write("##contig=<ID=chr%s,length=%d>\n" % (chrom, f.header.contigs[chrom].length))
    fw.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tC57_6J_DBA_2J\n")
    
    for x in variants:
        FS = x.filter
        if len(FS) == 1 and FS[0] == "PASS":
            FS = []
    
        ref = x.ref
        a1, a2 = x.allele1, x.allele2
            
        if len(x.overlaps) == 0 and a1 == a2 and a1 != ref:
            a1 = ref
            if len(FS) > 0:
                records_failed.append(x)
        else:
            records_failed.append(x)
            continue
        
        alleles = [ref]
        if a1 not in alleles:
            alleles.append(a1)
        if a2 not in alleles:
            alleles.append(a2)
        gt = "%s|%s" % (alleles.index(a1), alleles.index(a2))
        
        if len(FS) == 0:
            FS = ["PASS"]
        line = "\t".join(map(str, [x.chrom, x.start + 1, ".", x.ref, ",".join(alleles[1:]), ".", ";".join(FS), ".", "GT:PS", gt + ":0"]))
        fw.write(line + "\n")
        
! bgzip -f {outfile}
! tabix -f -p vcf {outfile}.gz

In [20]:
with open("stratifications/GRCm38_C57BL_6J_DBA_2J_SNP_Indel_failed.bed", "w+") as fw:
    for x in records_failed:
        fw.write("\t".join(map(str, [x.chrom, x.start, x.end])) + "\n")

In [22]:
! cat stratifications/GRCm38_C57BL_6J_DBA_2J_SNP_Indel_failed.bed | sort -k1,1 -k2,2n \
    | bedtools slop -i - -g {sizes} -b 50 \
    | bedtools merge -d 50 -i - > stratifications/GRCm38_C57BL_6J_DBA_2J_SNP_Indel_failed_slop50bp.bed

In [24]:
for path in glob.glob("stratifications/*.bed"):
    ! sort -k1,1 -k2,2n {path} | bedtools merge -i - | bgzip -c > {path}.gz
    ! tabix -f -p bed {path}.gz
    ! rm {path}

In [8]:
! zcat \
    stratifications/GRCm38_C57BL_6J_DBA_2J_SNP_Indel_failed_slop50bp.bed.gz \
    stratifications/GRCm38_DBA_SV_slop500bp.bed.gz \
    stratifications/GRCm38_LowComplexity_gt200bp_slop50bp.bed.gz \
    stratifications/GRCm38_SimpleRepeats_gt200bp_slop50bp.bed.gz \
    stratifications/GRCm38_TandemRepeats_gt200bp_slop50bp.bed.gz \
    stratifications/GRCm38_DBA_ExtremeRegion_200bp_slop200bp.bed.gz \
    stratifications/GRCm38_DBA_ExtremeRegion_500bp_slop500bp.bed.gz \
    stratifications/GRCm38_DBA_ExtremeRegion_1000bp_slop1000bp.bed.gz \
    | grep -v chrM | sort -k1,1 -k2,2n | bedtools merge -i - -d 50 \
    | bedtools complement -i - -g {sizes} | bgzip -c > GRCm38_C57BL_6J_DBA_2J_SNP_Indel.bed.gz
! tabix -f -p bed GRCm38_C57BL_6J_DBA_2J_SNP_Indel.bed.gz

## 2. C57BL/6NJ X DBA/2J

In general, the homozygous genome hold identical haplotypes, and variants does not overlap with any other variants at diploid. 

Therefore, the variants that overlap with other variants are considered located at comlex regions and will be ignored in benchmark.

In [3]:
variants1 = load_variants("data/C57BL_6NJ.mgp.v5.snps.dbSNP142.vcf.gz", "C57BL_6NJ", "snp")
variants2 = load_variants("data/C57BL_6NJ.mgp.v5.indels.dbSNP142.normed.vcf.gz", "C57BL_6NJ", "indel")
variants3 = load_variants("data/DBA_2J.mgp.v5.snps.dbSNP142.vcf.gz", "DBA_2J", "snp")
variants4 = load_variants("data/DBA_2J.mgp.v5.indels.dbSNP142.normed.vcf.gz", "DBA_2J", "indel")

In [4]:
variants_c57 = list(sorted(variants1 + variants2))
variants_dba = list(sorted(variants3 + variants4))
variants_c57 = scan_self_overlap(variants_c57)
variants_dba = scan_self_overlap(variants_dba)

The homozygous variants from C57 only or DBA only were combined as heterouzygou variants at F1 genome. 

For the homozygous variants that overlap between C57 and DBA, we combined them depending on the characteristics of the variants.

Low quality SNPs is challenge at benchmark, which will underestimate the performance. Therefore, we marked those regions contain low-quality SNPs as blacklist regions.

In [5]:
def collapse_variants_to_left(variants1, variants2):
    blacklist = [] # complex variants on F1
    results1 = [] # only on variants1
    results2 = [] # common on variants1 and variants2
    loader = ShiftLoader(variants2)
    for x in variants1:
        hits = [x2 for x2 in loader.fetch(obj=x)]
        if len(x.overlaps) == 0 and x.filter[0] == "PASS":
            if len(hits) == 0:
                results1.append(x)
            elif len(hits) == 1:
                x2 = hits[0]
                if len(x2.overlaps) == 0 and x2.filter[0] == "PASS" and x.type == x2.type:
                    if x.type == "snp":
                        results2.append([x, x2])
                    else:
                        if x.start == x2.start:
                            if len(x) == 1 and len(x2) == 1: # insertion
                                results2.append([x, x2])
                            elif len(x) > 1 and len(x2) > 1 and len(x) == len(x2): # deletion
                                results2.append([x, x2])
                            else:
                                blacklist.append(x)
                        else:
                            blacklist.append(x)
                else:
                    blacklist.append(x)
            else:
                blacklist.append(x)
        else:
            blacklist.append(x)
    blacklist = [[x.chrom, x.start, x.end] for x in blacklist]
    return results1, results2, blacklist

results1_1, results1_2, blacklist1 = collapse_variants_to_left(variants_c57, variants_dba)
results2_1, results2_2, blacklist2 = collapse_variants_to_left(variants_dba, variants_c57)

In [6]:
print("-" * 40)
print("For C57 variants:")
print("Variants on C57 only:", len(results1_1))
print("Variants on C57 and DBA:", len(results1_2))
print("Variants on blacklist:", len(blacklist1))
print("-" * 40)
print("For DBA variants:")
print("Variants on DBA only:", len(results2_1))
print("Variants on C57 and DBA:", len(results2_2))
print("Variants on blacklist:", len(blacklist2))

----------------------------------------
For C57 variants:
Variants on C57 only: 10135
Variants on C57 and DBA: 16246
Variants on blacklist: 140093
----------------------------------------
For DBA variants:
Variants on DBA only: 6095643
Variants on C57 and DBA: 16246
Variants on blacklist: 854001


In [7]:
array = []
for x in results1_1:
    assert x.allele1 == x.allele2
    array.append([x.chrom, x.start + 1, x.ref, (x.allele1, x.ref)])
for x in results2_1:
    assert x.allele1 == x.allele2
    array.append([x.chrom, x.start + 1, x.ref, (x.ref, x.allele1)])
for x, x2 in results1_2:
    assert x.allele1 == x.allele2
    assert x2.allele1 == x2.allele2
    assert x.type == x2.type
    array.append([x.chrom, x.start + 1, x.ref, (x.allele1, x2.allele1)])
array.sort()

In [11]:
infile = "data/C57BL_6NJ.mgp.v5.snps.dbSNP142.vcf.gz"
outfile = "GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel.vcf"

with pysam.VariantFile(infile) as f, open(outfile, "w+") as fw:
    fw.write("##fileformat=VCFv4.2\n")
    fw.write("##FORMAT=<ID=GT,Number=1,Type=String,Description=\"Genotype\">\n")
    fw.write("##FORMAT=<ID=PS,Number=1,Type=String,Description=\"Phase set for GT\">\n")
    for chrom in f.header.contigs:
        if chrom == "MT":
            continue
        fw.write("##contig=<ID=chr%s,length=%d>\n" % (chrom, f.header.contigs[chrom].length))
    fw.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tC57_6NJ_DBA_2J\n")

    for chrom, pos, ref, (pat, mat) in array:
        alleles = [ref]
        if pat not in alleles:
            alleles.append(pat)
        if mat not in alleles:
            alleles.append(mat)
        gt = "%s|%s" % (alleles.index(pat), alleles.index(mat))
        line = "\t".join(map(str, [chrom, pos, ".", ref, ",".join(alleles[1:]), ".", "PASS", ".", "GT:PS", gt + ":0"]))
        fw.write(line + "\n")
        
! bgzip -f {outfile}
! tabix -f -p vcf {outfile}.gz

In [12]:
with open("stratifications/GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel_failed.bed", "w+") as fw:
    for r in (blacklist1 + blacklist2):
        fw.write("\t".join(map(str, r)) + "\n")

In [15]:
! cat stratifications/GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel_failed.bed | sort -k1,1 -k2,2n \
    | bedtools slop -i - -g {sizes} -b 50 \
    | bedtools merge -d 50 -i - > stratifications/GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel_failed_slop50bp.bed

In [6]:
for path in glob.glob("stratifications/*.bed"):
    ! sort -k1,1 -k2,2n {path} | bedtools merge -i - | bgzip -c > {path}.gz
    ! tabix -f -p bed {path}.gz
    ! rm {path}

In [7]:
! zcat \
    stratifications/GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel_failed_slop50bp.bed.gz \
    stratifications/GRCm38_DBA_SV_slop500bp.bed.gz \
    stratifications/GRCm38_LowComplexity_gt200bp_slop50bp.bed.gz \
    stratifications/GRCm38_SimpleRepeats_gt200bp_slop50bp.bed.gz \
    stratifications/GRCm38_TandemRepeats_gt200bp_slop50bp.bed.gz \
    stratifications/GRCm38_DBA_ExtremeRegion_200bp_slop200bp.bed.gz \
    stratifications/GRCm38_DBA_ExtremeRegion_500bp_slop500bp.bed.gz \
    stratifications/GRCm38_DBA_ExtremeRegion_1000bp_slop1000bp.bed.gz \
    | grep -v chrM | sort -k1,1 -k2,2n | bedtools merge -i - -d 50 \
    | bedtools complement -i - -g {sizes} | bgzip -c > GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel.bed.gz
! tabix -f -p bed GRCm38_C57BL_6NJ_DBA_2J_SNP_Indel.bed.gz